In [ ]:
"""
forecast_project.py

Comprehensive project script for:
- Generating / Loading multivariate time series data
- Preprocessing: missing values, scaling, lag features
- Models: ARIMA baseline (univariate), PyTorch LSTM, Attention-LSTM, Transformer
- Expanding-window cross-validation for hyperparameter tuning
- Evaluation: MAE, RMSE, MAPE
- Attention interpretation utilities

NOTE: This script is written to be understandable and modifiable. For heavy training
you should run on a machine with GPU and adjust hyperparameters accordingly.

Dependencies:
- numpy, pandas, scikit-learn, torch, matplotlib, tqdm
- statsmodels (for ARIMA baseline) [optional but recommended]
- joblib (optional for saving objects)

Run example:
python forecast_project.py --data /path/to/synthetic_multivariate_timeseries.csv --target sensor_1
"""

import os
import argparse
import json
from datetime import timedelta
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Optional import for ARIMA baseline
try:
    import statsmodels.api as sm
    from statsmodels.tsa.arima.model import ARIMA
    STATSMODELS_AVAILABLE = True
except Exception:
    STATSMODELS_AVAILABLE = False

# ------------------------
# Utilities
# ------------------------
def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return float("nan")
    return (np.abs((y_true - y_pred)[mask] / y_true[mask])).mean() * 100

# ------------------------
# Data pipeline
# ------------------------
def load_and_preprocess(path, target_col, lags=[1,24,168], test_size=0.2, val_size=0.1):
    """
    Loads CSV with a timestamp column named 'timestamp' and numeric feature columns.
    Performs:
    - parse dates
    - forward-fill/backfill for missing values
    - scaling (StandardScaler)
    - creates lag features for all numeric columns
    - splits into train/val/test by time
    Returns dict with scalers and arrays.
    """
    df = pd.read_csv(path, parse_dates=["timestamp"])
    df = df.sort_values("timestamp").reset_index(drop=True)
    # identify numeric columns (excluding timestamp)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if target_col not in numeric_cols:
        raise ValueError(f"target_col '{target_col}' not numeric or not present.")
    # missing value handling
    df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')
    # create lag features
    for lag in lags:
        for c in numeric_cols:
            df[f"{c}_lag_{lag}"] = df[c].shift(lag)
    df = df.dropna().reset_index(drop=True)
    # feature columns
    lag_cols = [c for c in df.columns if "_lag_" in c]
    # Additional time features
    df["hour"] = df["timestamp"].dt.hour
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    # Choose features (include some external covariates if present)
    feature_cols = lag_cols.copy()
    # keep temperature / holiday if present
    if "temperature" in df.columns:
        feature_cols.append("temperature")
    if "is_holiday" in df.columns:
        feature_cols.append("is_holiday")
    feature_cols += ["hour", "dayofweek"]
    # scaling
    scaler = StandardScaler()
    df_features = scaler.fit_transform(df[feature_cols])
    X = df_features
    y = df[target_col].values.reshape(-1,1)
    # train/val/test split by time (chronological)
    n = len(df)
    test_n = int(n * test_size)
    val_n = int(n * val_size)
    train_end = n - test_n - val_n
    X_train = X[:train_end]
    y_train = y[:train_end]
    X_val = X[train_end:train_end+val_n]
    y_val = y[train_end:train_end+val_n]
    X_test = X[train_end+val_n:]
    y_test = y[train_end+val_n:]
    return {
        "X_train": X_train, "y_train": y_train,
        "X_val": X_val, "y_val": y_val,
        "X_test": X_test, "y_test": y_test,
        "scaler": scaler, "feature_cols": feature_cols,
        "df_processed": df  # returning for ARIMA indexing convenience
    }

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ------------------------
# Models
# ------------------------
class LSTMForecaster(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.2):
        super().__init__()
        # We'll treat input as a single time-step sequence with many features (feature-engineered)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim
    def forward(self, x):
        # x shape: (batch, features) -> reshape to (batch, seq_len=1, features)
        x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

class AttentionLayer(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.W = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.u = nn.Linear(hidden_dim, 1, bias=False)
    def forward(self, H):
        # H: (batch, seq_len, hidden_dim)
        score = torch.tanh(self.W(H))  # (batch, seq_len, hidden_dim)
        score = self.u(score).squeeze(-1)  # (batch, seq_len)
        alpha = torch.softmax(score, dim=1).unsqueeze(-1)  # (batch, seq_len, 1)
        context = (H * alpha).sum(dim=1)  # (batch, hidden_dim)
        return context, alpha

class AttentionLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout, bidirectional=False)
        self.att = AttentionLayer(hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1)
    def forward(self, x):
        x = x.unsqueeze(1)  # seq_len=1
        H, _ = self.lstm(x)  # (batch, seq_len, hidden_dim)
        context, alpha = self.att(H)
        out = self.fc(context)
        return out, alpha

class SimpleTransformerForecaster(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_encoder_layers=2, dim_feedforward=128, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        self.fc = nn.Linear(d_model, 1)
    def forward(self, x):
        x = x.unsqueeze(1)  # seq_len=1
        x = self.input_proj(x)
        h = self.transformer(x)
        out = self.fc(h[:, -1, :])
        return out

# ------------------------
# Training / Evaluation
# ------------------------
def train_model(model, optimizer, criterion, train_loader, val_loader=None, epochs=10, device='cpu'):
    model.to(device)
    best_val_loss = float('inf')
    best_state = None
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            # model may return (out, alpha)
            if isinstance(out, tuple):
                pred = out[0]
            else:
                pred = out
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        avg_train = np.mean(train_losses) if train_losses else float('nan')
        if val_loader is not None:
            model.eval()
            val_losses = []
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(device), yb.to(device)
                    out = model(xb)
                    pred = out[0] if isinstance(out, tuple) else out
                    loss = criterion(pred, yb)
                    val_losses.append(loss.item())
            avg_val = np.mean(val_losses) if val_losses else float('nan')
            if avg_val < best_val_loss:
                best_val_loss = avg_val
                best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
        else:
            avg_val = None
        print(f"Epoch {epoch+1}/{epochs} - train_loss: {avg_train:.6f} - val_loss: {avg_val}")
    if best_state:
        model.load_state_dict(best_state)
    return model

def evaluate_model(model, X, y, batch_size=128, device='cpu'):
    model.to(device)
    model.eval()
    ds = TimeSeriesDataset(X, y)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=False)
    preds = []
    trues = []
    alphas = []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            out = model(xb)
            if isinstance(out, tuple):
                pred, alpha = out
                alphas.append(alpha.cpu().numpy())
            else:
                pred = out
            preds.append(pred.cpu().numpy())
            trues.append(yb.numpy())
    y_pred = np.vstack(preds).ravel()
    y_true = np.vstack(trues).ravel()
    return y_true, y_pred, alphas

# ------------------------
# ARIMA baseline
# ------------------------
def arima_baseline(series, train_end_index, forecast_horizon):
    """
    Trains a simple ARIMA on the training portion of `series` (1-D numpy array) and forecasts next `forecast_horizon` points.
    Returns series_pred (for the test period).
    Requires statsmodels.
    """
    if not STATSMODELS_AVAILABLE:
        raise RuntimeError("statsmodels not installed. Install statsmodels to run ARIMA baseline.")
    train = series[:train_end_index]
    model = ARIMA(train, order=(5,1,0))
    res = model.fit()
    preds = res.forecast(steps=forecast_horizon)
    return preds

# ------------------------
# Cross-validation (expanding window)
# ------------------------
def expanding_window_cv(X, y, model_builder, param_dict, n_splits=3, initial_train_size=None, val_size=500, device='cpu'):
    """
    Simple expanding-window cross-validation to tune hyperparameters in `param_dict` (a dict).
    model_builder is a function that accepts **params and returns an uninitialized PyTorch model.
    Returns best_params and history.
    """
    from itertools import product
    keys = list(param_dict.keys())
    grid = [dict(zip(keys, vals)) for vals in product(*param_dict.values())]
    best = None
    best_score = float('inf')
    history = []
    n = len(X)
    if initial_train_size is None:
        initial_train_size = int(n * 0.5)
    for params in grid:
        scores = []
        for split in range(n_splits):
            train_end = initial_train_size + split * val_size
            val_start = train_end
            val_end = train_end + val_size
            if val_end > n:
                break
            X_train = X[:train_end]
            y_train = y[:train_end]
            X_val = X[val_start:val_end]
            y_val = y[val_start:val_end]
            # build model
            model = model_builder(**params)
            optimizer = torch.optim.Adam(model.parameters(), lr=params.get("lr", 1e-3), weight_decay=params.get("weight_decay", 0.0))
            criterion = nn.MSELoss()
            train_ds = TimeSeriesDataset(X_train, y_train)
            val_ds = TimeSeriesDataset(X_val, y_val)
            train_loader = DataLoader(train_ds, batch_size=params.get("batch_size", 256), shuffle=True)
            val_loader = DataLoader(val_ds, batch_size=params.get("batch_size", 256), shuffle=False)
            model = train_model(model, optimizer, criterion, train_loader, val_loader=val_loader, epochs=params.get("epochs", 5), device=device)
            y_val_true, y_val_pred, _ = evaluate_model(model, X_val, y_val, batch_size=256, device=device)
            score = rmse(y_val_true, y_val_pred)
            scores.append(score)
        avg_score = np.mean(scores) if scores else float('inf')
        history.append((params, avg_score))
        if avg_score < best_score:
            best_score = avg_score
            best = params
    return best, history

# ------------------------
# Main runner
# ------------------------
def main(args):
    print("Loading and preprocessing data...")
    data = load_and_preprocess(args.data, target_col=args.target, lags=[1,24,168], test_size=0.2, val_size=0.1)
    X_train, y_train = data["X_train"], data["y_train"]
    X_val, y_val = data["X_val"], data["y_val"]
    X_test, y_test = data["X_test"], data["y_test"]
    input_dim = X_train.shape[1]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device:", device)
    # Baseline ARIMA (univariate) - using the original series (not scaled) for interpretability
    print("Running ARIMA baseline (univariate) ...")
    if STATSMODELS_AVAILABLE:
        # Need original series without lagging and scaling; reload raw series from CSV
        df = pd.read_csv(args.data, parse_dates=["timestamp"])
        df = df.sort_values("timestamp").reset_index(drop=True)
        # For simplicity, we'll use last len(X_test) points as test horizon
        total_len = len(df)
        horizon = len(X_test)
        try:
            arima_preds = arima_baseline(df[args.target].values, total_len - horizon, horizon)
            arima_rmse = rmse(df[args.target].values[total_len - horizon:], arima_preds)
            print(f"ARIMA RMSE: {arima_rmse:.4f}")
        except Exception as e:
            print("ARIMA training failed:", e)
            arima_preds = None
    else:
        print("statsmodels not available; skipping ARIMA baseline. Install statsmodels to enable this baseline.")
        arima_preds = None

    # Standard LSTM
    print("Training standard LSTM ...")
    lstm_params = {"input_dim": input_dim, "hidden_dim": 64, "num_layers": 2, "dropout": 0.2}
    lstm = LSTMForecaster(**lstm_params)
    optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-3, weight_decay=1e-5)
    criterion = nn.MSELoss()
    train_loader = DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=256, shuffle=True)
    val_loader = DataLoader(TimeSeriesDataset(X_val, y_val), batch_size=256, shuffle=False)
    lstm = train_model(lstm, optimizer, criterion, train_loader, val_loader=val_loader, epochs=10, device=device)
    y_true_lstm, y_pred_lstm, _ = evaluate_model(lstm, X_test, y_test, batch_size=256, device=device)
    print("LSTM metrics - RMSE: %.4f MAE: %.4f MAPE: %.4f" % (rmse(y_true_lstm, y_pred_lstm), mae(y_true_lstm, y_pred_lstm), mape(y_true_lstm, y_pred_lstm)))

    # Attention LSTM
    print("Training Attention-LSTM ...")
    att_params = {"input_dim": input_dim, "hidden_dim": 64, "num_layers": 1, "dropout": 0.1}
    att_model = AttentionLSTM(**att_params)
    optimizer = torch.optim.Adam(att_model.parameters(), lr=5e-4, weight_decay=1e-5)
    att_model = train_model(att_model, optimizer, criterion, train_loader, val_loader=val_loader, epochs=10, device=device)
    y_true_att, y_pred_att, alphas = evaluate_model(att_model, X_test, y_test, batch_size=256, device=device)
    print("Attention LSTM metrics - RMSE: %.4f MAE: %.4f MAPE: %.4f" % (rmse(y_true_att, y_pred_att), mae(y_true_att, y_pred_att), mape(y_true_att, y_pred_att)))

    # Transformer
    print("Training Transformer ...")
    trans_params = {"input_dim": input_dim, "d_model": 64, "nhead": 4, "num_encoder_layers": 2, "dim_feedforward": 128}
    trans_model = SimpleTransformerForecaster(**trans_params)
    optimizer = torch.optim.Adam(trans_model.parameters(), lr=1e-3, weight_decay=1e-5)
    trans_model = train_model(trans_model, optimizer, criterion, train_loader, val_loader=val_loader, epochs=10, device=device)
    y_true_tr, y_pred_tr, _ = evaluate_model(trans_model, X_test, y_test, batch_size=256, device=device)
    print("Transformer metrics - RMSE: %.4f MAE: %.4f MAPE: %.4f" % (rmse(y_true_tr, y_pred_tr), mae(y_true_tr, y_pred_tr), mape(y_true_tr, y_pred_tr)))

    # Summarize metrics
    results = {
        "ARIMA": {
            "preds": None if arima_preds is None else arima_preds.tolist(),
            "rmse": None if arima_preds is None else float(rmse(df[args.target].values[total_len - horizon:], arima_preds)),
            "mae": None if arima_preds is None else float(mae(df[args.target].values[total_len - horizon:], arima_preds)),
            "mape": None if arima_preds is None else float(mape(df[args.target].values[total_len - horizon:], arima_preds))
        },
        "LSTM": {
            "rmse": float(rmse(y_true_lstm, y_pred_lstm)),
            "mae": float(mae(y_true_lstm, y_pred_lstm)),
            "mape": float(mape(y_true_lstm, y_pred_lstm))
        },
        "AttentionLSTM": {
            "rmse": float(rmse(y_true_att, y_pred_att)),
            "mae": float(mae(y_true_att, y_pred_att)),
            "mape": float(mape(y_true_att, y_pred_att))
        },
        "Transformer": {
            "rmse": float(rmse(y_true_tr, y_pred_tr)),
            "mae": float(mae(y_true_tr, y_pred_tr)),
            "mape": float(mape(y_true_tr, y_pred_tr))
        }
    }
    out_path = args.output or "."
    os.makedirs(out_path, exist_ok=True)
    with open(os.path.join(out_path, "results_summary.json"), "w") as f:
        json.dump(results, f, indent=2)
    print("Results saved to", os.path.join(out_path, "results_summary.json"))

    # Attention interpretation: if `alphas` collected, average them
    if alphas:
        # alphas is a list of arrays per batch; concatenate and average across samples
        all_alphas = np.vstack([a.squeeze(1) for a in alphas])
        mean_alpha = np.mean(all_alphas, axis=0)  # shape (seq_len,)
        # save
        np.savetxt(os.path.join(out_path, "attention_mean.txt"), mean_alpha)
        print("Saved mean attention weights to attention_mean.txt")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, default="synthetic_multivariate_timeseries.csv", help="Path to CSV file")
    parser.add_argument("--target", type=str, default="sensor_1", help="Target column name")
    parser.add_argument("--output", type=str, default="./output", help="Output directory")
    args = parser.parse_args()
    main(args)
